In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd

import sys
sys.path.append("../")
import os
from src.functional import free_gpu_cache

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.models import load_LM
import torch
from nnsight import LanguageModel
from src.utils import env_utils

# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

lm = load_LM(
    model_key=model_key,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 30/30 [01:00<00:00,  2.02s/it]

loaded /home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-70B-Instruct | size: 36650.535 MB


In [4]:
from src.dataset import SampleV3, DatasetV3
from src.functional import predict_next_token, get_hs 

sample_1 = SampleV3(
    protagonist="Nora",
    perpetrator="Jon",
    objects=["milk", "juice"],
    containers=["pitcher", "mug"],
    event_idx=None,
    event_noticed=False
)

print(f"{sample_1.true_state=}")
print(f"{sample_1.protagonist_belief=}")

sample_2 = SampleV3(
    protagonist="Nora",
    perpetrator="Jon",
    objects=["juice", "milk"],
    containers=["pitcher", "mug"],
    event_idx=None,
    event_noticed=False
)

print(f"{sample_2.true_state=}")
print(f"{sample_2.protagonist_belief=}")

dataset = DatasetV3(samples = [sample_1, sample_2])

sample_1.true_state={'pitcher': 'milk', 'mug': 'juice'}
sample_1.protagonist_belief={'pitcher': 'milk', 'mug': 'juice'}
sample_2.true_state={'pitcher': 'juice', 'mug': 'milk'}
sample_2.protagonist_belief={'pitcher': 'juice', 'mug': 'milk'}


In [14]:
prompt_1, answer_1 = dataset.__getitem__(
    0, 
    set_actor="protagonist",
    # set_ans="no",
    set_container=0,
    set_obj=1
)
print(prompt_1)
print(answer_1)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer:" tag.

Story: Nora is working in a busy restaurant. A customer asks Nora for milk. Nora grabs an opaque pitcher and fills it with milk. Then Nora grabs another opaque mug and fills it with juice.
Question: Does Nora believe the pitcher contains juice?
Answer:
no


[PredictedToken(token=' No', prob=0.5644365549087524, logit=22.046875, token_id=2360),
 PredictedToken(token=' no', prob=0.22103652358055115, logit=21.109375, token_id=912),
 PredictedToken(token=' NO', prob=0.1920398771762848, logit=20.96875, token_id=5782),
 PredictedToken(token=' **', prob=0.014130393043160439, logit=18.359375, token_id=3146),
 PredictedToken(token='No', prob=0.0035173327196389437, logit=16.96875, token_id=2822)]

In [15]:
prompt_2, answer_2 = dataset.__getitem__(
    1, 
    set_actor="protagonist",
    # set_ans="no",
    set_container=0,
    set_obj=0
)
print(prompt_2)
print(answer_2)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer:" tag.

Story: Nora is working in a busy restaurant. A customer asks Nora for juice. Nora grabs an opaque pitcher and fills it with juice. Then Nora grabs another opaque mug and fills it with milk.
Question: Does Nora believe the pitcher contains juice?
Answer:
yes


[PredictedToken(token=' YES', prob=0.6052390336990356, logit=23.1875, token_id=14410),
 PredictedToken(token=' yes', prob=0.2483895868062973, logit=22.296875, token_id=10035),
 PredictedToken(token=' Yes', prob=0.13933393359184265, logit=21.71875, token_id=7566),
 PredictedToken(token='Yes', prob=0.0024351258762180805, logit=17.671875, token_id=9642),
 PredictedToken(token='yes', prob=0.0015969945816323161, logit=17.25, token_id=9891)]

In [16]:
from src.functional import prepare_input, find_token_range

interested_word = "juice"

input_1 = prepare_input(
    prompts = prompt_1,
    tokenizer=lm,
    return_offsets_mapping=True
)
token_pos_1 = find_token_range(
    string=prompt_1,
    substring=interested_word,
    occurrence=-2,
    tokenizer=lm,
    offset_mapping=input_1["offset_mapping"][0]
)[1] - 1 # last token position
print(token_pos_1, lm.tokenizer.decode(input_1["input_ids"][0][token_pos_1]))


input_2 = prepare_input(
    prompts = prompt_2,
    tokenizer=lm,
    return_offsets_mapping=True
)
token_pos_2 = find_token_range(
    string=prompt_2,
    substring=interested_word,
    occurrence=-2,
    tokenizer=lm,
    offset_mapping=input_2["offset_mapping"][0]
)[1] - 1 # last token position
print(token_pos_2, lm.tokenizer.decode(input_2["input_ids"][0][token_pos_2]))

92  juice
80  juice


In [20]:
predict_next_token(lm, input_1)

[PredictedToken(token=' No', prob=0.5644365549087524, logit=22.046875, token_id=2360),
 PredictedToken(token=' no', prob=0.22103652358055115, logit=21.109375, token_id=912),
 PredictedToken(token=' NO', prob=0.1920398771762848, logit=20.96875, token_id=5782),
 PredictedToken(token=' **', prob=0.014130393043160439, logit=18.359375, token_id=3146),
 PredictedToken(token='No', prob=0.0035173327196389437, logit=16.96875, token_id=2822)]

In [21]:
predict_next_token(lm, input_2)

[PredictedToken(token=' YES', prob=0.6052390336990356, logit=23.1875, token_id=14410),
 PredictedToken(token=' yes', prob=0.2483895868062973, logit=22.296875, token_id=10035),
 PredictedToken(token=' Yes', prob=0.13933393359184265, logit=21.71875, token_id=7566),
 PredictedToken(token='Yes', prob=0.0024351258762180805, logit=17.671875, token_id=9642),
 PredictedToken(token='yes', prob=0.0015969945816323161, logit=17.25, token_id=9891)]

In [17]:
from src.functional import get_hs, free_gpu_cache, PatchSpec

free_gpu_cache()
layer_name_format = "model.layers.{}"
layers = [
    layer_name_format.format(i) 
    for i in range(0, 10, 1)
]

hs_1 = get_hs(
    lm=lm, input=input_1,
    locations = [(l, token_pos_1) for l in layers],
)
hs_2 = get_hs(
    lm=lm, input=input_2,
    locations = [(l, token_pos_2) for l in layers],
)

In [19]:
patches = []

for layer in layers:
    patches.append(
        PatchSpec(
            location=(layer, token_pos_2),
            patch=hs_1[(layer, token_pos_1)],
        )
    )
    pred = predict_next_token(lm, input_2, patches=patches, k=2)[0]
    print(f"{layer} => {pred}")

model.layers.0 => " YES" (p=0.605)
model.layers.1 => " YES" (p=0.605)
model.layers.2 => " YES" (p=0.608)
model.layers.3 => " YES" (p=0.611)
model.layers.4 => " YES" (p=0.609)
model.layers.5 => " YES" (p=0.614)
model.layers.6 => " YES" (p=0.616)
model.layers.7 => " YES" (p=0.616)
model.layers.8 => " YES" (p=0.622)
model.layers.9 => " YES" (p=0.625)
